In [8]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
from sklearn import preprocessing
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import train_test_split
import datetime
from sklearn.model_selection import GridSearchCV

In [9]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [10]:
df_ciudades = df_train.groupby('ciudad').agg({'metrostotales':'mean'})
df_ciudades2 = df_test.groupby('ciudad').agg({'metrostotales':'mean'})

df_ciudades2.reset_index(inplace = True)
ciudades_list2 = df_ciudades2.values.tolist()

df_ciudades.reset_index(inplace = True)
ciudades_list = df_ciudades.values.tolist()

only_ciudades_train = []
only_ciudades_test = []
for ciudades in ciudades_list:
    only_ciudades_train.append(ciudades[0])

for ciudades in ciudades_list2:
    only_ciudades_test.append(ciudades[0])
    
print(len(only_ciudades_train))
print(len(only_ciudades_test))

dif = []

for ciudad in only_ciudades_test:
    if ciudad not in only_ciudades_train:
        dif.append(ciudad)
        
for ciudad in dif:
    ciudades_list.append([ciudad,'nan'])

#print(ciudades_list)
        
for i in range (0,len(ciudades_list)):
    df_train.loc[df_train.ciudad == ciudades_list[i][0], 'ciudad'] = float(ciudades_list[i][1])
    df_test.loc[df_test.ciudad == ciudades_list[i][0], 'ciudad'] = float(ciudades_list[i][1])
    
df_train.apply(lambda col:pd.to_numeric(col, errors='coerce'))
df_test.apply(lambda col:pd.to_numeric(col, errors='coerce'))
df_train.astype({'ciudad': 'float64'})
df_test.astype({'ciudad': 'float64'})

df_train['ciudad'] = pd.to_numeric(df_train['ciudad'], downcast='float')
df_test['ciudad'] = pd.to_numeric(df_test['ciudad'], downcast='float')

df_train.dtypes

875
576


id                              int64
titulo                         object
descripcion                    object
tipodepropiedad                object
direccion                      object
ciudad                        float32
provincia                      object
antiguedad                    float64
habitaciones                  float64
garages                       float64
banos                         float64
metroscubiertos               float64
metrostotales                 float64
idzona                        float64
lat                           float64
lng                           float64
fecha                          object
gimnasio                      float64
usosmultiples                 float64
piscina                       float64
escuelascercanas              float64
centroscomercialescercanos    float64
precio                        float64
dtype: object

In [11]:
#----------------PREPROCESAMIENTO----------------

df_train = df_train.drop(['fecha','titulo','descripcion','direccion','idzona','lat','lng'], axis=1)
df_test = df_test.drop(['fecha','titulo','descripcion','direccion','idzona','lat','lng'], axis=1)
df_train = df_train.fillna(value = {'provincia': 0, 'tipodepropiedad': 0})

#XGBoost maneja los NaNs, ver despues si lo dejamos como esta

df_train = pd.get_dummies(df_train, drop_first=True)#, sparse = True)
df_test = pd.get_dummies(df_test, drop_first=True)#, sparse = True)

#print(df_train_xgboost.shape)
#precio = df_train_xgboost['precio']
#df_train_xgboost.drop(labels=['precio'], axis=1,inplace = True)
#df_train_xgboost.insert(len(df_train_xgboost.columns), 'Precio', precio)

#X_train, y_train = df_train_xgboost.iloc[:,:-1],df_train_xgboost.iloc[:,-1]

X = df_train.drop("precio", axis=1)
y = df_train['precio']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)

#df_test_xgboost['tipodepropiedad_Garage'] = 0
#df_test_xgboost['tipodepropiedad_Hospedaje'] = 0
#df_test_xgboost = df_test_xgboost[list(X_train.columns.values)]

xgb1 = XGBRegressor()

#Ir cambiando parametros para ver como functiona

parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [0.04], #probamos con 0.03,0.04,0.05,0.06,0.07
              'max_depth': [9], #probamos con 5,6,7,8,9,10,11
              'min_child_weight': [1], #Probamos con 1,2,3,4,5,6
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7], #probamos 0.3,0.4,0.5,0.6,0.7,0.8
              'n_estimators': [700],  #Se probo con 400,500,600,700,800
            'gamma': [0]} #Se probo con 0,1,5,10

#Estos son sin correccion de metros
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7214816192533363
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 700, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7218556040564428
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.04, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 700, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7219040830119198
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.04, 'max_depth': 9, 'min_child_weight': 4, 'n_estimators': 700, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7242695855881088
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.04, 'max_depth': 9, 'min_child_weight': 2, 'n_estimators': 700, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7245128843441556
#Con {'colsample_bytree': 0.7, 'learning_rate': 0.04, 'max_depth': 9, 'min_child_weight': 1, 'n_estimators': 700, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7} da 0.7246402143520269

#Con fechas nos da 0.7438760200186147

#Con las ciudades nos dio 0.7706985066967957

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X,
         y)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   2 | elapsed:  4.6min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   2 out of   2 | elapsed:  4.6min finished
c:\users\manuel\appdata\local\programs\python\python37-32\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
c:\users\manuel\appdata\local\programs\python\python37-32\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


0.7706985066967957
{'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.04, 'max_depth': 9, 'min_child_weight': 1, 'n_estimators': 700, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}
